In [1]:
import os
import langchain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.sql_database import SQLDatabase
from langchain.utilities import SQLDatabase

In [2]:
# connecting database
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database_name = os.getenv("DB_NAME")

connection_params = {"host": host, "user": user,
                         "password": password, "port": port, "database": database_name}

db = SQLDatabase.from_uri(
    f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}"
)

In [3]:
# uses SQLDatabase class as it provides a get_table_info method that can be used to get column information as well as sample data from the table.(not recommended)
# To mitigate risk of leaking sensitive data, limit permissions to read and scope to the tables that are needed.
# creates SQL queries but doesn't run it
chain= create_sql_query_chain(llm=ChatOpenAI(temperature=0,model="gpt-3.5-turbo-0613"),db=db)
sql_query=chain.invoke({"question":"how many tables do i have in my database?"})
print(sql_query)

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public';


In [4]:
# executing the sql query genertaed against the db object
db.run(sql_query)

'[(26,)]'

In [5]:
langchain.debug=True # to see how langchain operates and interacts with our opeanai model while executing our queries 
sql_query=chain.invoke({"question":"which age group has the most views?"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "which age group has the most views?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,top_k,table_info>] Entering Chain run with input:
{
  "question": "which age group has the most views?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,top_k,table_info> > 3:chain:<lambda>] Entering Chain run with input:
{
  "question": "which age group has the most views?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,top_k,table_info> > 3:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": 5
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,top_k,table_info> > 4:chain:<lambda>] Entering Chain run with input:
{
  "question": "which age group has the most views?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,top_k,table_info> > 4:chain:<lambda>] s] Exiting Chain run with o

In [6]:
db.run(sql_query)

"[('25–34 years', 72.7)]"

In [7]:
# creating an sql agent
langchain.debug=False
agent_executor= create_sql_agent(
    llm=ChatOpenAI(temperature=0,model="gpt-3.5-turbo-0613"),
    toolkit=SQLDatabaseToolkit(db=db,llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIFunctionsAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(


In [8]:
print(agent_executor.run("which gender has the highest average view duration"))

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


cities_chart_data, cities_table_data, content_type_chart_data, content_type_table_data, device_type_chart_data, device_type_table_data, geography_chart_data, geography_table_data, new_and_returning_viewers_chart_data, new_and_returning_viewers_table_data, operating_system_chart_data, operating_system_table_data, sharing_service_chart_data, sharing_service_table_data, subscription_source_chart_data, subscription_source_table_data, subscription_status_chart_data, subscription_status_table_data, subtitles_and_cc_chart_data, subtitles_and_cc_table_data, traffic_source_chart_data, traffic_source_table_data, viewer_age_table_data, viewer_gender_table_data, viewership_by_date_table_data, views_by_date

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Invoking: `sql_db_schema` with `viewer_gender_table_data`



CREATE TABLE viewer_gender_table_data (
	"Viewer_gender" TEXT, 
	"Views_in_percent" DOUBLE PRECISION, 
	"Average_view_duration" TIME WITHOUT TIME ZONE, 
	"Average_percentage_viewed_in_percent" DOUBLE PRECISION, 
	"Watch_time_hours_in_percent" DOUBLE PRECISION
)

/*
3 rows from viewer_gender_table_data table:
Viewer_gender	Views_in_percent	Average_view_duration	Average_percentage_viewed_in_percent	Watch_time_hours_in_percent
Female	20.16	00:05:23	9.36	23.38
Male	79.84	00:04:27	7.83	76.62
*/

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Based on the data from the `viewer_gender_table_data` table, the gender with the highest average view duration is female. The average view duration for females is 00:05:23, while for males it is 00:04:27.

> Finished chain.
Based on the data from the `viewer_gender_table_data` table, the gender with the highest average view duration is female. The average view duration for females is 00:05:23, while for males it is 00:04:27.


In [9]:
# it corrects it self when it counters errors
print(agent_executor.run("which date has the most view"))

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


cities_chart_data, cities_table_data, content_type_chart_data, content_type_table_data, device_type_chart_data, device_type_table_data, geography_chart_data, geography_table_data, new_and_returning_viewers_chart_data, new_and_returning_viewers_table_data, operating_system_chart_data, operating_system_table_data, sharing_service_chart_data, sharing_service_table_data, subscription_source_chart_data, subscription_source_table_data, subscription_status_chart_data, subscription_status_table_data, subtitles_and_cc_chart_data, subtitles_and_cc_table_data, traffic_source_chart_data, traffic_source_table_data, viewer_age_table_data, viewer_gender_table_data, viewership_by_date_table_data, views_by_date

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Invoking: `sql_db_schema` with `views_by_date`



CREATE TABLE views_by_date (
	"Date" DATE, 
	"Views" BIGINT
)

/*
3 rows from views_by_date table:
Date	Views
2020-06-28	1
2020-06-29	72
2020-06-30	76
*/

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Invoking: `sql_db_query` with `SELECT Date, Views FROM views_by_date ORDER BY Views DESC LIMIT 1`
responded: Based on the schema of the `views_by_date` table, we can query the date with the most views. Let me construct the query for you.

Error: (psycopg2.errors.UndefinedColumn) column "date" does not exist
LINE 1: SELECT Date, Views FROM views_by_date ORDER BY Views DESC LI...
               ^
HINT:  Perhaps you meant to reference the column "views_by_date.Date".

[SQL: SELECT Date, Views FROM views_by_date ORDER BY Views DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/f405)

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Invoking: `sql_db_query` with `SELECT "Date", "Views" FROM views_by_date ORDER BY "Views" DESC LIMIT 1`
responded: Apologies for the error. It seems that the column name "Date" is not recognized. Let me correct the query for you.

[(datetime.date(2022, 9, 26), 367)]

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The date with the most views is September 26, 2022, with a total of 367 views.

> Finished chain.
The date with the most views is September 26, 2022, with a total of 367 views.


In [12]:
print(agent_executor.run("how many views do i get from android operating system"))

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


cities_chart_data, cities_table_data, content_type_chart_data, content_type_table_data, device_type_chart_data, device_type_table_data, geography_chart_data, geography_table_data, new_and_returning_viewers_chart_data, new_and_returning_viewers_table_data, operating_system_chart_data, operating_system_table_data, sharing_service_chart_data, sharing_service_table_data, subscription_source_chart_data, subscription_source_table_data, subscription_status_chart_data, subscription_status_table_data, subtitles_and_cc_chart_data, subtitles_and_cc_table_data, traffic_source_chart_data, traffic_source_table_data, viewer_age_table_data, viewer_gender_table_data, viewership_by_date_table_data, views_by_date

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Invoking: `sql_db_schema` with `operating_system_table_data`



CREATE TABLE operating_system_table_data (
	"Operating_system" TEXT, 
	"Views" BIGINT, 
	"Watch_time_in_hours" DOUBLE PRECISION, 
	"Average_view_duration" TIME WITHOUT TIME ZONE
)

/*
3 rows from operating_system_table_data table:
Operating_system	Views	Watch_time_in_hours	Average_view_duration
Total	26625	2157.3497	00:04:51
Windows	12506	1099.1683	00:05:16
Android	6089	381.0323	00:03:45
*/

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...


SELECT "Views" FROM operating_system_table_data WHERE "Operating_system" = 'Android' LIMIT 10

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Invoking: `sql_db_query` with `SELECT "Views" FROM operating_system_table_data WHERE "Operating_system" = 'Android' LIMIT 10`


[(6089,)]

/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/kanu/redashvenv1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


You have received a total of 6,089 views from the Android operating system.

> Finished chain.
You have received a total of 6,089 views from the Android operating system.
